In [2]:
!python -m jupytools export -nb "11_resnet_two_sites_same.ipynb" -o .

Exported: 11_resnet_two_sites_same.ipynb -> resnet_two_sites_same.py
1 notebook(s) exported into folder: .


In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# https://www.kaggle.com/tanlikesmath/rcic-fastai-starter

## Imports 

In [3]:
#export
from collections import OrderedDict
import json
import os
from os.path import dirname, join
from functools import reduce
from pdb import set_trace

import cv2 as cv
import jupytools
import jupytools.syspath
import numpy as np
import pandas as pd
import PIL.Image
import matplotlib.pyplot as plt

from catalyst.utils import get_one_hot
import pretrainedmodels
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from visdom import Visdom

jupytools.syspath.add(join(dirname(os.getcwd()), 'protein_project'))
jupytools.syspath.add('rxrx1-utils')
if jupytools.is_notebook():
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm as tdqm
    
import contextlib, io
with contextlib.redirect_stderr(io.StringIO()):
    from basedir import ROOT, NUM_CLASSES
    from catalyst_visdom import BatchMetricsPlotCallback, EpochMetricsPlotCallback
    from dataset import load_data, build_stats_index, RxRxDataset
    from model.utils import freeze_all, unfreeze_layers
    from visual import rgb, six, show_1, show

In [4]:
#export
torch.set_default_tensor_type(torch.FloatTensor)

## Dataset Reader

In [5]:
#export
from augmentation import JoinChannels, SwapChannels, Resize, ToFloat, Rescale
from augmentation import VerticalFlip, HorizontalFlip, PixelStatsNorm, composer
from augmentation import AugmentedImages

In [6]:
#export
class RxRxImages(Dataset):
    def __init__(self, meta_df, img_dir, site=1, channels=(1, 2, 3, 4, 5, 6), 
                 open_image=PIL.Image.open, n_classes=NUM_CLASSES, train=True):
        
        self.records = meta_df.to_records(index=False)
        self.img_dir = img_dir
        self.site = site
        self.channels = channels
        self.n = len(self.records)
        self.open_image = open_image
        self.n_classes = n_classes
        self.train = train
        
    def _get_image_path(self, index, channel):
        r = self.records[index]
        exp, plate, well = r.experiment, r.plate, r.well
        subdir = 'train' if self.train else 'test'
        path = f'{self.img_dir}/{subdir}/{exp}/Plate{plate}/{well}_s{self.site}_w{channel}.png'
        return path
    
    def __getitem__(self, index):
        paths = [self._get_image_path(index, ch) for ch in self.channels]
        img = np.stack([self.open_image(p) for p in paths])
        img = img.astype(np.float32)
        img = img.transpose(1, 2, 0)
        r = self.records[index]
        if self.train:
            sirna = r.sirna
            target = int(sirna)
            onehot = get_one_hot(target, num_classes=self.n_classes)
            return {'features': img, 'targets': target, 'targets_one_hot': onehot}
        else:
            id_code = r.id_code
            return {'features': img, 'id_code': id_code}
    
    def __len__(self): 
        return self.n

In [7]:
#export
class TwoSiteImages(Dataset):
    def __init__(self, ds1, ds2):
        assert len(ds1) == len(ds2)
        self.ds1, self.ds2 = ds1, ds2
        self.size = len(ds1)
        
    def __getitem__(self, index):
        return {'site1': self.ds1[index], 'site2': self.ds2[index]}
    
    def __len__(self):
        return self.size

## Pipeline Preparation

In [8]:
#export
from split import StratifiedSplit
splitter = StratifiedSplit()
trn_df, val_df = splitter(pd.read_csv(ROOT/'train.csv')) 
tst_df = pd.read_csv(ROOT/'test.csv')

In [9]:
#export
sz = 512
trn_ds = TwoSiteImages(
    ds1=AugmentedImages(ds=RxRxImages(trn_df, ROOT, site=1), tr=composer(resize=sz)),
    ds2=AugmentedImages(ds=RxRxImages(trn_df, ROOT, site=2), tr=composer(resize=sz))
)
val_ds = TwoSiteImages(
    ds1=AugmentedImages(ds=RxRxImages(val_df, ROOT, site=1), tr=composer(resize=sz)),
    ds2=AugmentedImages(ds=RxRxImages(val_df, ROOT, site=2), tr=composer(resize=sz))
)
tst_ds = TwoSiteImages(
    ds1=AugmentedImages(ds=RxRxImages(tst_df, ROOT, site=1, train=False), tr=composer(resize=sz)),
    ds2=AugmentedImages(ds=RxRxImages(tst_df, ROOT, site=2, train=False), tr=composer(resize=sz))
)

In [10]:
#export
def new_loader(ds, bs, shuffle=True, num_workers=12):
    return DataLoader(ds, batch_size=bs, shuffle=shuffle, num_workers=num_workers)

## Model

In [11]:
#export
def resnet18(head_fn=None, n_classes=NUM_CLASSES):
    model_fn = pretrainedmodels.__dict__['resnet18']
    model = model_fn(num_classes=1000, pretrained='imagenet')
    new_conv = nn.Conv2d(6, 64, 7, 2, 3, bias=False)
    conv1 = model.conv1.weight
    with torch.no_grad():
        new_conv.weight[:, :] = torch.stack([torch.mean(conv1, 1)]*6, dim=1)
    model.conv1 = new_conv
    return model

In [12]:
#export
from catalyst.contrib.modules import GlobalConcatPool2d
class ResNet18_TwoSites(nn.Module):
    def __init__(self, n_classes=NUM_CLASSES):
        super().__init__()
        
        base = resnet18()
        feat_dim = base.last_linear.in_features
        
        self.base = base 
        self.pool = GlobalConcatPool2d()
        self.head = nn.Sequential(
            nn.Linear(feat_dim * 2, feat_dim * 2),
            nn.Dropout(0.2),
            nn.BatchNorm1d(feat_dim * 2),
            nn.ReLU(inplace=True),
            nn.Linear(feat_dim * 2, n_classes)
        )
        
    def forward(self, s1, s2):
        f1 = self.base.features(s1)
        f2 = self.base.features(s2)
        f_merged = self.pool(f1 + f2)
        out = self.head(f_merged.squeeze())
        return out

In [13]:
#export
def freeze_all(model):
    for name, child in model.named_children():
        print('Freezing layer:', name)
        for param in child.parameters():
            param.requires_grad = False

In [14]:
#export
def unfreeze_all(model):
    for name, child in model.named_children():
        print('Un-freezing layer:', name)
        for param in child.parameters():
            param.requires_grad = True

In [15]:
#export
def unfreeze(model, names):
    for name, child in model.named_children():
        if name not in names:
            continue
        print('Un-freezing layer:', name)
        for param in child.parameters():
            param.requires_grad = True

## Train

In [16]:
#export
model = ResNet18_TwoSites()
freeze_all(model)

Freezing layer: base
Freezing layer: pool
Freezing layer: head


In [17]:
#export
from visdom import Visdom

In [18]:
#export
class RollingLoss:
    def __init__(self, smooth=0.98):
        self.smooth = smooth
        self.prev = 0
    def __call__(self, curr, batch_no):
        a = self.smooth
        avg_loss = a*self.prev + (1 - a)*curr
        debias_loss = avg_loss/(1 - a**batch_no)
        self.prev = avg_loss
        return debias_loss

In [19]:
#export
def create_loaders(batch_size):
    trn_dl = new_loader(trn_ds, bs=batch_size, shuffle=True)
    val_dl = new_loader(val_ds, bs=batch_size, shuffle=False)
    return OrderedDict([('train', trn_dl), ('valid', val_dl)])

In [20]:
#export
loss_fn = nn.CrossEntropyLoss()
device = torch.device('cuda:1')

In [21]:
# sched = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
#     torch.optim.AdamW(model.parameters(), lr=3e-4),
#     T_0=1000, T_mult=2, eta_min=1e-5
# )
# lrs = []
# for _ in range(1000 + 2000 + 4000):
#     lr = sched.get_lr()
#     lrs.extend(lr)
#     sched.step()
# plt.plot(lrs)

In [ ]:
#export
epochs = 1000
patience = 500

opt = torch.optim.AdamW(model.parameters(), lr=3e-4)
model = model.to(device)
rolling_loss = dict(train=RollingLoss(), valid=RollingLoss())
steps = dict(train=0, valid=0)

trials = 0
best_metric = -np.inf
history = []
stop = False

vis = Visdom(server='0.0.0.0', port=9090,
             username=os.environ['VISDOM_USERNAME'],
             password=os.environ['VISDOM_PASSWORD'])

loaders = create_loaders(batch_size=40)

sched = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    opt, T_0=len(loaders['train']), T_mult=2, eta_min=1e-5
)

for epoch in range(1, epochs+1):
    print(f'Epoch [{epoch}/{epochs}]')
    
    if epoch == 1:
        unfreeze_layers(model, ['head'])
    elif epoch == 5:
        unfreeze_all(model)
    
    #curr_lr = opt.param_groups[0]['lr']
    #vis.line(X=[epoch], Y=[curr_lr], win='lr', name='lr', update='append')    
    iteration = dict(epoch=epoch, train_loss=list(), valid_loss=list())
    #print(f"Current learning rate: {opt.param_groups[0]['lr']:.2E}")
    
    for name, loader in loaders.items():
        is_training = name == 'train'
        count = 0
        metric = 0.0
        
        with torch.set_grad_enabled(is_training):
            for batch in loader:
                steps[name] += 1
                opt.zero_grad()

                y = batch['site1']['targets'].to(device)
                
                out = model(
                    batch['site1']['features'].to(device),
                    batch['site2']['features'].to(device)
                )
                
                if is_training:
                    loss = loss_fn(out, y)
                    loss.backward()
                    opt.step()
                    sched.step()
                    
                    curr_lr = opt.param_groups[0]['lr']
                    vis.line(X=[steps[name]], Y=[curr_lr], win='lr', name='lr', update='append')    
                
                avg_loss = rolling_loss[name](loss.item(), steps[name])
                iteration[f'{name}_loss'].append(avg_loss)
                y_pred = out.softmax(dim=1).argmax(dim=1)
                acc = (y_pred == y).float().mean().item()
                metric += acc
                count += len(batch)
                vis.line(X=[steps[name]], Y=[avg_loss], name=f'{name}_loss', 
                         win=f'{name}_loss', update='append', 
                         opts=dict(title=f'Running Loss [{name}]'))
        
        metric /= count
        iteration[f'{name}_acc'] = metric
        vis.line(X=[epoch], Y=[avg_loss], name=f'{name}', win='avg_loss',
                 update='append', opts=dict(title='Average Epoch Loss'))
        vis.line(X=[epoch], Y=[metric], name=f'{name}', win='accuracy', 
                 update='append', opts=dict(title=f'Accuracy'))
        
        last_loss = iteration[f'{name}_loss'][-1]
        
        print(f'{name} metrics: accuracy={metric:2.3%}, loss={last_loss:.4f}')
          
        if is_training:
            # sched.step()
            pass
          
        else:
            # sched.step(metric)
            if metric > best_metric:
                trials = 0
                best_metric = metric
                print('Score improved!')
                torch.save(model.state_dict(), f'two_sites_one_base/train.{epoch}.pth')
            else:
                trials += 1
                if trials >= patience:
                    stop = True
                    break
    
    history.append(iteration)
    
    print('-' * 80)
    
    if stop:
        print(f'Early stopping on epoch: {epoch}')
        break

torch.save(history, 'history.pth')

## Test

In [16]:
device = torch.device('cuda:0')
model = ResNet18_TwoSites()
freeze_all(model)
model = model.to(device)
state = torch.load('two_sites_one_base/train.30.pth', map_location=lambda loc, storage: loc)
model.load_state_dict(state)
_ = model.eval()

Freezing layer: base
Freezing layer: pool
Freezing layer: head


In [18]:
with torch.no_grad():
    test_dl = new_loader(tst_ds, bs=64)
    preds = {}
    for batch in tqdm(test_dl):
        s1 = batch['site1']['features']
        s2 = batch['site2']['features']
        out = model(s1.to(device), s2.to(device))
        y_hat = out.argmax(dim=-1).cpu().numpy()
        preds.update(dict(zip(batch['site1']['id_code'], y_hat.astype(int))))

In [19]:
preds_df = pd.DataFrame([
    {'id_code': id_code, 'sirna': sirna} 
    for id_code, sirna in preds.items()])
preds_df.head(3)

,id_code,sirna
0,HUVEC-20_4_G02,983
1,HUVEC-22_4_I20,778
2,HUVEC-22_3_M18,145


In [20]:
len(preds_df)

19897

In [21]:
preds_df.to_csv('submit.csv', index=False, columns=['id_code', 'sirna'])

In [22]:
from IPython.display import FileLink
FileLink('submit.csv')

/home/ck/code/tasks/protein/submit.csv